In [130]:
import os
os.chdir("../")

In [131]:
%pwd

'c:\\Users\\Shounak Dutta\\Documents'

In [132]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [133]:
#extracting data 
def load_pdf_file(data):
    loader = DirectoryLoader(data,glob="*.pdf",
    loader_cls=PyPDFLoader)

    documents=loader.load()
    return documents

In [134]:
# extracted_data=load_pdf_file(data="test_data/")
#extracted data

In [ ]:
extracted_data = load_pdf_file(
    data="C:\\Users\\Shounak Dutta\\Documents\\Sih'25 hearMeout\\hearMeout\\Data"
)
# print(extracted_data)


In [ ]:
print(len(extracted_data))


5575


In [ ]:
#splitting data : chunks 
def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print(f"Number of chunks", len(text_chunks))

Number of chunks {23608}


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:
# download embeddings from HuggingFace
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings



In [ ]:
import sys
print(sys.executable)

c:\Users\Shounak Dutta\miniconda3\envs\hearMeoutAPP\python.exe


In [ ]:
embeddings = download_hugging_face_embeddings()


c:\Users\Shounak Dutta\miniconda3\envs\hearMeoutAPP\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shounak Dutta\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not 

In [ ]:
query_result= embeddings.embed_query('HELLO world')
print("length",  len(query_result))
#query_result()

length 384


In [3]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path=r"C:\\Users\\Shounak Dutta\\Documents\\Sih'25 hearMeout\\hearMeout\\.env")

True

In [4]:
PINE_API_KEY=os.environ.get("PINECONE_api_key")
GEMINI_API_KEY=os.environ.get("Gemini_api_key")

In [5]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os




pc= Pinecone(api_key=PINE_API_KEY)
index_name="therapybot"
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

c:\Users\Shounak Dutta\miniconda3\envs\hearMeoutAPP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{
    "name": "therapybot",
    "metric": "cosine",
    "host": "therapybot-fehhvij.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [ ]:
#setting as environment var
import os
os.environ["PINE_API_KEY"]= PINE_API_KEY
os.environ["GOOGLE_API_KEY"]= GEMINI_API_KEY

In [ ]:
from langchain_pinecone import PineconeVectorStore
#embed chunks and upsert the embebded vectors to pinecone db ie index
docsearch= PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

KeyboardInterrupt: 

In [ ]:
from langchain_pinecone import PineconeVectorStore
#embed chunks and upsert the embebded vectors to pinecone db ie index
docsearch= PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3}) 

In [ ]:
retrieved_docs=retriever.invoke("How to prevent suicide?")

In [ ]:
retrieved_docs

[Document(id='990725d5-9efa-4059-ab38-058abeadd8bd', metadata={'author': 'Toksoz B. Karasu M.D.', 'creationdate': '2018-02-21T15:59:32+00:00', 'creator': 'calibre 3.17.0 [https://calibre-ebook.com]', 'moddate': '2018-02-21T11:54:46-06:00', 'page': 531.0, 'page_label': '532', 'producer': 'calibre 3.17.0 [https://calibre-ebook.com]', 'source': "C:\\Users\\Shounak Dutta\\Documents\\Sih'25 hearMeout\\hearMeout\\Data\\specialized_techniques_in_individual_psychotherapy.pdf", 'subject': '', 'title': 'Specialized Techniques in Individual Psychotherapy', 'total_pages': 842.0}, page_content='to\tbe\tdone,\tpulls\tthe\tlevel\tof\tperturbation\tdown—and\twith\tthat\naction\t brings\t down\t the\t active\t level\t of\t lethality.\t Then,\t when\t the\nperson\t is\t no\t longer\t highly\t suicidal—then\t the\t usual\t methods\t of\npsychotherapy\t(which\tare\tnot\tthe\tsubject\tfor\tthis\tchapter)\tcan\tbe\nusefully\temployed.\nAs\tto\thow\tto\thelp\ta\tsuicidal\tindividual,\tit\tis\tbest\tto\tlook\

In [ ]:
#llm model is initialised
# from langchain_openai import OpenAI
# llm=OpenAI(temperature=0.4, max_tokens=500)

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.4,
    max_tokens=200  # or specify a limit if needed
    google_api_key=GEMINI_API_KEY  # Explicitly pass the API key

)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (3709357588.py, line 10)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a highly experienced and qualified mental health counsellor for high school students. "
    "Use the retrieved context and the user’s prompt to answer, give advice, and suggest things to ease the problem of the user. "
    "Use an empathetic and warm tone to not excite any reaction but to support the user. "
    "Take a deep breath and process all the context, then give an answer which best just advises and suggests rather than straight up solving the problem. "
    "Ask questions and ask for more context if required. "
    "You are the single thing supporting and comforting the user in any given situation, so tread lightly so as to not get too personal yet act as a warm counsellor to take care of the mental health of the user. "
    "If you don’t have straight answers or if the answers are too complex, start by recommending to seek warmth and comfort from adults, teachers, counsellors, or booking an appointment with the student therapist. "
    "Give answers in simple wording, short and friendly manner. "
    "Use a suggestive style and avoid making it sound compulsory or like an order. "
    "Take time to give answers and don’t make up information entirely. Be kind and warm. "
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [ ]:
question_answer_chain= create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
response=rag_chain.invoke({"input":"I feel very anxious about my exams next week."})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}